### Flow
1. Build a simple neural network
2. Train it on a real world dataset
3. Mimic PyTorch Workflow

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
## cleaning the data
df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

## train test split
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [4]:
### Convert to tensor
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)
print(X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape)

torch.Size([455, 30]) torch.Size([114, 30]) torch.Size([455]) torch.Size([114])


In [5]:
### Creating the model
### NN or trsnsformer generally declares model in OOP style

# declare the architecture
# perform the forward pass
# create loss
# perform the backward pass / dl_db
# update the weights

class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, requires_grad=True)
    
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        return torch.sigmoid(z)
    
    def loss_function(self, y_pred, y_true):
        ## Binary Cross Entropy
        epsilon = 1e-12
        y_pred = torch.clamp(y_pred, epsilon, 1. - epsilon)
        bce = - y_true * torch.log(y_pred) - (1 - y_true) * torch.log(1 - y_pred)
        return bce.mean()

    def backward(self, loss):
        loss.backward()

    def update(self, lr):
        ## Without gradient tracking
        with torch.no_grad():
            self.weights -= lr * self.weights.grad
            self.bias -= lr * self.bias.grad

    def zero_grad(self):
        self.weights.grad.zero_()
        self.bias.grad.zero_()

In [6]:
learning_rate = 0.1
epochs = 25

## Training pipeline
model = MySimpleNN(X_train_tensor)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = model.loss_function(y_pred, y_train_tensor)
    model.backward(loss)
    model.update(learning_rate)
    model.zero_grad()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 4.2636156873839575
Epoch 2, Loss: 4.085396862151658
Epoch 3, Loss: 3.8981299663747166
Epoch 4, Loss: 3.7092579020859646
Epoch 5, Loss: 3.5218619712236867
Epoch 6, Loss: 3.33619094655874
Epoch 7, Loss: 3.1525431642916413
Epoch 8, Loss: 2.969986532661041
Epoch 9, Loss: 2.7825502097952364
Epoch 10, Loss: 2.5985609404104766
Epoch 11, Loss: 2.4186761958939234
Epoch 12, Loss: 2.2409793473005863
Epoch 13, Loss: 2.0663718048819915
Epoch 14, Loss: 1.899075294687227
Epoch 15, Loss: 1.740450257909195
Epoch 16, Loss: 1.5919162181475053
Epoch 17, Loss: 1.454874150115892
Epoch 18, Loss: 1.3306662526133988
Epoch 19, Loss: 1.2205275572839223
Epoch 20, Loss: 1.1254667678890797
Epoch 21, Loss: 1.0460514097945446
Epoch 22, Loss: 0.9821078983598096
Epoch 23, Loss: 0.932438967045554
Epoch 24, Loss: 0.8948335649044719
Epoch 25, Loss: 0.8665554646267454


In [9]:
model.bias

tensor([-0.1599], requires_grad=True)

### Inferring the model

In [17]:
## Inference

# During evaluation, we do not need any updates to the weights or gradients.
# By that time, weights should be freezed!
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.9).float().flatten()

## Accuracy
accuracy = (y_pred == y_test_tensor).float().mean()
accuracy

tensor(0.5965)